In [3]:
# Import packages
import warnings
%matplotlib inline
warnings.simplefilter("ignore")
from IPython.display import Image
from ipyleaflet import *
import ee
import numpy as np
import matplotlib as mp

In [5]:
# Initialize
ee.Initialize()

# Matplotlib inline
%matplotlib inline

# Zoom
#mpld3.enable_notebook()

### 2 Define Functions
2.0 Surface Reflectance

### def SR(img):
    img = img.addBands(img.select(['blue', 'green', 'red', 'nir', 'swir1', 'swir2']).multiply(0.0001))
    img = img.addBands(img.select(['tir']).multiply(0.1));
    img = img.select(['blue_1', 'green_1', 'red_1', 'nir_1', 'swir1_1', 'swir2_1','tir_1'],['blue', 'green', 'red', 'nir', 'swir1', 'swir2','tir'])
    
    return img

### 2.1 Indices

In [8]:
def NDVI(img):
    img = img.addBands(img.normalizedDifference(['nir', 'red']).rename('ndvi'))
    img = img.addBands(img.normalizedDifference(['green', 'nir']).rename('ndwi'))
    img = img.addBands(img.normalizedDifference(['green', 'swir1']).rename('mndwi'))
    img = img.addBands(img.normalizedDifference(['nir', 'swir1']).rename('ndmi'))
    img = img.addBands(img.normalizedDifference(['nir', 'swir2']).rename('nbr'))
    return img

### 2.2 Cloud Mask

In [9]:
def NDCI(img):
    img = img.addBands(img.select('tir').unitScale(240,270)); 
    img = img.addBands(img.normalizedDifference(['tir_1','swir2']))
    img = img.addBands(img.select('nd').gt(0.4).rename('clear'));
    img = img.addBands(img.select('nd').lte(0.4).rename('cloud'));
    mask = ee.Image(1).updateMask(img.select('cloud')).byte().reduceToVectors(
        {
            geometry: aoi, 
            crs: 'EPSG:3005', 
            scale: 300, 
            maxPixels: 1e13
        }
    )
      
    maskImg = ee.Image(1).subtract(ee.Image(1).clipToCollection(mask).unmask())
    
    return img.updateMask(maskImg)

### 3. Band Names

In [10]:
BANDS_MSS = ['B4', 'B4', 'B4', 'B5', 'B6', 'B6', 'B6', 'B6'];
BANDS_TM  = ['B1', 'B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B6'];
BANDS_ETM = ['B1', 'B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B6'];
BANDS_OLI = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10'];
BANDS_MSI = ['B1', 'B2', 'B3', 'B4', 'B8', 'B11','B12','B12'];
BANDS_AST = ['B01', 'B01', 'B01', 'B02', 'B3N', 'B04', 'B07', 'B13'];
NAMES_STD = ['blue2', 'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'tir',];

### 4. Area of Interest

In [12]:
nor = 56.862497
eas = -121.638316

aoi_gee = {
    'type':'Point', 
    'coordinates':[eas,nor]};

aoi_json = {
    "type": "Feature",
    "geometry": {"type": "Point",
        "coordinates": [eas, nor]}}

print ('GEE', aoi_gee, 'JSON', aoi_json)

GEE {'type': 'Point', 'coordinates': [-121.638316, 56.862497]} JSON {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-121.638316, 56.862497]}}


### 5. Collections

In [14]:
# Merge collections, normalize band names
col = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").select(BANDS_OLI, NAMES_STD)
col = col.merge(ee.ImageCollection("LANDSAT/LE07/C01/T1_SR").select(BANDS_ETM, NAMES_STD))
col = col.merge(ee.ImageCollection("LANDSAT/LT05/C01/T1_SR").select(BANDS_TM, NAMES_STD))
col = col.merge(ee.ImageCollection("LANDSAT/LT04/C01/T1_SR").select(BANDS_TM, NAMES_STD))

# Filter collection
col = col.filterDate('1985-01-01','2020-01-01')
col = col.filterMetadata("CLOUD_COVER", "less_than", 30)
#col = col.filter(ee.Filter.dayOfYear(100,250))
col = col.filterBounds(aoi_gee)

print ("There are ", col.size().getInfo(), " images in the collection.")

There are  487  images in the collection.


### 6. First Image

In [16]:
# Select first image
image = col.first()

# Correct surface reflectance 
image = SR(image)

# Calculate indices
image = NDVI(image)

print ("The first image is:", image.get('LANDSAT_ID').getInfo())
print ("The bands are:", image.bandNames().getInfo())

The first image is: LC08_L1TP_048020_20130717_20170309_01_T1
The bands are: ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'tir', 'ndvi', 'ndwi', 'mndwi', 'ndmi', 'nbr']


### 6.2 Display first image

In [17]:
thumbnail = image.getThumbUrl({'bands': 'swir1,nir,red', 'min':0,'max':0.4,'size':'800'})
Image(url=thumbnail)

In [19]:
# Get a download URL for an image.
path = image.select(['swir1','nir','red','ndvi']).getDownloadUrl({'scale': 30, 'crs': 'EPSG:4326'})
print (path)

https://earthengine.googleapis.com/api/download?docid=a90dde88c2542a8ad5f3e975dd1cca0c&token=3de335077863e36ecf34c0cf72a7bfa1


In [21]:
#Retrieve information over image collection
ts_values = col.getRegion(aoi,500).getInfo()

NameError: name 'aoi' is not defined

In [22]:
# Reshape Landsat 8
header = ts_values[0]
data = array(ts_values[1:])
iTime = header.index('time')
time = [datetime.datetime.fromtimestamp(i/1000) for i in (data[0:,iTime].astype(float))]

# List of used image bands
band_list = ['nir',u'red']

iBands = [header.index(b) for b in band_list]

yData = data[0:,iBands].astype(np.float)

# Calculate NDVI
red = yData[:,0]
nir = yData[:,1]
ndvi = (nir - red) / (nir + red)

# Reshape NDVI array into Pandas Dataframe
df = pd.DataFrame(data=ndvi*-1, index=time, columns=['ndvi'])
df = df.dropna()

df_monthly = df.resample('M').min()

NameError: name 'ts_values' is not defined